In [ ]:
import torch
from ultralytics import YOLO
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))


# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch

model = YOLO("runs/detect/train53/weights/last.pt")

# Use the model
model.train(data="data.yaml", epochs=100, device= "0", batch= -1)  # train the model


## using trained model


In [ ]:
import os
from ultralytics import YOLO

# Load the model
model = YOLO('yoloModels/collab3_last.pt')

def process_images(input_folder, output_folder):
    """
    Run the YOLO model on all PNG images in the input folder and save the results in the output folder.

    :param input_folder: Path to the folder containing input PNG images.
    :param output_folder: Path to the folder to save output images.
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.png'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Run inference on the image
            results = model(input_path)


            # Process results and save the output image
            for result in results:
                result.save(filename=output_path)
                print(f"Processed and saved: {output_path}")

# Example usage
input_folder = "testTif"
output_folder = "test_colab3"
process_images(input_folder, output_folder)



initialize openAI api to read number

In [2]:
from openai import OpenAI
import os

from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("TMG_OpenAI_API"))


def readNumber (base64_image):

    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
        "role": "user",
        "content": [
            {"type": "text", "text": "This image might have a number in it. Can you tell me what number it is and only that number? if there is no number can you respond with no number. Numbers maybe rotated, if the numbers are rotated can you respond with no number?"},
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                
            },
            },
        ],
        }
    ],
    max_tokens=300,
    )

    # print(response.choices[0].message.content)
    return (response.choices[0].message.content)


In [3]:
import cv2
from ultralytics import YOLO
import base64

# Load the YOLO model
model = YOLO('yoloModels/colab_2_last.pt')

# Load the image
image_path = 'test/22028-D-254-0_page_1.png'
image = cv2.imread(image_path)

# Perform inference
results = model(image)
dimensions = []

def is_float(value):
    """ Check if the value is an integer. """
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def encode_image(image_path):
    _, buffer = cv2.imencode(".jpg", image_path)
    return base64.b64encode(buffer).decode('utf-8')

# View results
for r in results:
    boxes = r.boxes.cpu().numpy()    # print the Boxes object containing the detection bounding boxes

    XYcoords = boxes.xyxy
    dimClass = boxes.cls
    # print(XYcoords)
    # print(dimClass)

    # Iterate through the array of bounding boxes
    dimCount = 1
    for i, (xmin, ymin, xmax, ymax) in enumerate(XYcoords, 1):
        
        if dimClass[i-1] == 0:
        
            # Convert coordinates to integers (if they are not already)
            xmin, ymin, xmax, ymax = map(int, (xmin, ymin, xmax, ymax))

            # Crop the region of interest (ROI) from the image
            roi = image[ymin:ymax, xmin:xmax]

            # Resize image 
            resized_image = cv2.resize(roi, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
            # print("Image shape:", resized_image.shape)

            #do thresholding
            gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            _, thresholded_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                    #general label
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

            # # Define label position (slightly above the top-left corner of the bounding box)
            label_position_gen = (xmin, ymin - 10) if ymin - 10 > 10 else (xmin, ymin + 20)
            label_gen = f'{i}'
            print(label_gen)

            # Add label to the image
            cv2.putText(image, label_gen, label_position_gen, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # encode image
            encoded_img = encode_image(thresholded_image)

            # print the value that openai reads
            
            dim = readNumber(encoded_img)
            
            try:
            # Try to convert the result to a float
                number = float(dim)
                print(number)
                dimensions.append(number)  # Append to the array if successful
            except ValueError:
            # Handle the case where dim is not a number (e.g., it's a string)
                print(f"Value returned is not a number: {dim}")
                # dimensions.append(f"{label_gen}_n/a")

            # rotating the image

                rotatedImg = cv2.rotate(thresholded_image, cv2.ROTATE_90_CLOCKWISE)
                encoded_img_rot = encode_image(rotatedImg)

                dim_rotate = readNumber(encoded_img_rot)
                dimensions.append(dim_rotate)


            # Print or store the extracted text
            # print(f"Box {i}: {text.strip()}, Rotated: {text_rotated}")

    
            cv2.waitKey(0)  # Wait indefinitely until a key is pressed
    cv2.destroyAllWindows()  # Close the window
        

    # Save the annotated image
    output_path = 'labeled_image2.png'
    print(dimensions)
    cv2.imwrite(output_path, image)

    print(f"Image saved with labels as {output_path}")



    



0: 480x640 28 dimensions, 1 hole, 5 dias, 2 gd&ts, 48.0ms
Speed: 4.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
2
90.0
3
75.0
5
Value returned is not a number: no number
6
80.0
7
Value returned is not a number: No number
9
310.0
10
215.0
11
8.0
13
Value returned is not a number: No number
14
35.0
15
Value returned is not a number: No number.
16
Value returned is not a number: No number.
17
53.0
18
Value returned is not a number: No number
19
270.0
20
Value returned is not a number: No number.
21
Value returned is not a number: No number.
22
Value returned is not a number: No number.
23
125.0
24
136.0
25
Value returned is not a number: No number
26
Value returned is not a number: No number
28
Value returned is not a number: No number
30
Value returned is not a number: No number
31
170.0
32
190.5
33
Value returned is not a number: No number.
35
Value returned is not a number: No number.
[90.0, 75.0, '440', 80.0, 'No number.', 310.0, 215.0, 8.0,